# The Impact of Ambient Temperature on Server Efficiency


Hypothesis: Server power consumption increases as temperature increases reducing server efficiency. As PUE values approach 1 an increasing portion of the Data centre's power is used in the server therfore there is likely to be a trade-off on operating temperature depending on cooling infrastructure and number of servers in the datacenter. 

------

Plan:

- Load in all of the SERT results avoiding any invalid ones
- Merge data as needed 
- Generate graphs showing power consumption against load and temperature
- Find a trade-off between operating temperature and number of servers. 


In [1]:
import numpy as np
import pandas as pd
import re
import os.path
import glob
from parse_results import process_results_xml

In [2]:
target_dir = 'OneDrive//Results - PowerEdge'


In [3]:
metrics_data = pd.DataFrame()
test_details = pd.DataFrame()
scores = pd.DataFrame()

for f in glob.glob(f'{target_dir}//**//results.xml', recursive=True):
    try:
        # Name of test directory -- sert-xxxx
        test_name = os.path.basename(os.path.dirname(f))
        
        if os.path.isfile(f'{target_dir}//{test_name}//invalid.png'):
            print(test_name, ': INVALID RESULT')
            continue
        
        metrics, score, env = process_results_xml(f)
        
        file_df = pd.DataFrame.from_records(metrics)
        # Remove calibration runs but record the calibration score against each loadlevel to calculate actual loadlevel
        calibrations = file_df.loc[file_df['loadlevel']=='calibration', ['worklet', 'score']]
        calibrations = calibrations.rename(columns={'score': 'calibration_score'})
        
        file_df = pd.merge(file_df.drop(index=calibrations.index), calibrations, how='left', on='worklet')
        file_df['actual_load'] = file_df['score'] / file_df['calibration_score']
        file_df['test-name'] = test_name
        
        score_df = pd.DataFrame.from_records(score)
        score_df['test-name'] = test_name
    
        metrics_data = metrics_data.append(file_df, ignore_index=True)
        test_details = test_details.append(pd.DataFrame.from_records(env, index=[test_name]))
        scores = scores.append(score_df, ignore_index=True)
        
    except Exception as e:
        print(f, ': FAILED -- ', e)
        

sert-0129 : INVALID RESULT
sert-0077 : INVALID RESULT
sert-0056 : INVALID RESULT
sert-0131 : INVALID RESULT
sert-0099 : INVALID RESULT
sert-0096 : INVALID RESULT
sert-0035 : INVALID RESULT
sert-0111 : INVALID RESULT
sert-0108 : INVALID RESULT
sert-0094 : INVALID RESULT
sert-0041 : INVALID RESULT
sert-0107 : INVALID RESULT
sert-0070 : INVALID RESULT
sert-0092 : INVALID RESULT
sert-0132 : INVALID RESULT
sert-0143 : INVALID RESULT
sert-0027 : INVALID RESULT
sert-0105 : INVALID RESULT
OneDrive//Results - PowerEdge/sert-0101/results.xml : FAILED --  'NoneType' object has no attribute 'text'
sert-0128 : INVALID RESULT
sert-0034 : INVALID RESULT
sert-0036 : INVALID RESULT
sert-0073 : INVALID RESULT
sert-0082 : INVALID RESULT
sert-0104 : INVALID RESULT
sert-0098 : INVALID RESULT
sert-0033 : INVALID RESULT
sert-0095 : INVALID RESULT
sert-0059 : INVALID RESULT
sert-0031 : INVALID RESULT
OneDrive//Results - PowerEdge/sert-0093/results.xml : FAILED --  time data '2021-08-02T22:13:01+01:00' does no

In [4]:
metrics_data.to_csv('metrics.csv')
test_details.to_csv('test_details.csv')
scores.to_csv('scores.csv')

In [10]:
full_details = pd.DataFrame()
full_details = pd.merge(metrics_data, scores, how='left', on=['test-name', 'worklet', 'loadlevel', 'workload', 'score', 'watts-avg'])
full_details = pd.merge(full_details, test_details, left_on='test-name', right_index=True)

In [11]:
full_details.to_csv('all_data.csv')

In [12]:
full_details

,loadlevel,worklet,score,workload,watts-min,watts-max,watts-avg,start,end,temp-min,...,norm-score,ref-score,efficiency-score,cpu,dimm_size_mb,dimms,model,psu,ref,vendor
0,100%,Compress,4.552566e+04,CPU,405.50,413.90,409.045161,2021-08-07 00:47:01.528000+01:00,2021-08-07 00:48:02.009000+01:00,20.8750,...,6.574672,6924.40,16.073218,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R620-HighP-MidT,Dell Inc.
1,75%,Compress,3.413598e+04,CPU,359.31,364.29,361.495161,2021-08-07 00:48:14.282000+01:00,2021-08-07 00:49:14.600000+01:00,20.8125,...,4.929810,6924.40,13.637278,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R620-HighP-MidT,Dell Inc.
2,50%,Compress,2.279919e+04,CPU,300.37,303.47,301.838387,2021-08-07 00:49:27.129000+01:00,2021-08-07 00:50:27.646000+01:00,20.8750,...,3.292587,6924.40,10.908444,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R620-HighP-MidT,Dell Inc.
3,25%,Compress,1.137390e+04,CPU,252.48,255.46,253.712903,2021-08-07 00:50:40.100000+01:00,2021-08-07 00:51:40.353000+01:00,20.8125,...,1.642583,6924.40,6.474180,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R620-HighP-MidT,Dell Inc.
4,100%,CryptoAES,3.604070e+05,CPU,384.40,385.80,385.122581,2021-08-07 00:57:10.127000+01:00,2021-08-07 00:58:10.875000+01:00,20.8125,...,6.107754,59008.11,15.859247,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R620-HighP-MidT,Dell Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945,Flood_Full,Flood3,6.996335e+01,Memory,302.89,319.06,309.097273,2021-08-09 21:11:20.083000+01:00,2021-08-09 21:13:08.941000+01:00,21.8750,...,6.073207,11.52,19.648206,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R630-E52690v3-750W-Eff_BIOS,Dell Inc.
2946,Flood_Half,Flood3,7.019133e+01,Memory,310.31,324.40,315.324091,2021-08-09 21:13:27.854000+01:00,2021-08-09 21:14:23.576000+01:00,21.8750,...,6.092998,11.52,19.322970,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R630-E52690v3-750W-Eff_BIOS,Dell Inc.
2947,Capacity3_Base,Capacity3,1.227797e+06,Memory,374.95,400.06,384.493226,2021-08-09 21:15:31.803000+01:00,2021-08-09 21:16:32.165000+01:00,21.8125,...,8.290591,148095.20,21.562385,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R630-E52690v3-750W-Eff_BIOS,Dell Inc.
2948,Capacity3_Max,Capacity3,3.358638e+06,Memory,382.06,396.21,384.063438,2021-08-09 21:16:44.446000+01:00,2021-08-09 21:17:44.764000+01:00,21.8125,...,22.678913,148095.20,59.049914,Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz,8192,8,PowerEdge R630,750,R630-E52690v3-750W-Eff_BIOS,Dell Inc.
